In [ ]:
#### Código principal ####

# Importar funciones y módulos requeridos
import os
import sys

# módulo que contiene todas las funciones desarrolladas y utilizadas en este código
sys.path.append('./my_module/')
import nimgenetics

# ----------------------------------- 3. SISTEMA, DISEÑO Y DESARROLLO ------------------------------------------- #

## Directorios importantes ##
# folder_path es la carpeta donde están ubicados todos los informes de NIMGenetics
# folder_result es la carpeta donde se guardarán:
# - 3.2: Los informes filtrados y renombrados
# - 3.3: Las imagenes en formato PNG
folder_path = 'path_to_folder_root'
folder_result = 'path_to_folder_result'

### 3.2 Obtención de informes clínicos de la base de datos de NIMGenetics ###

## 3.2.1 Mapeo de los informes localizados en la carpeta principal ##

# La función load_files es utilizada para mapear los informes (PDF) dentro de folder_path
# l_path contiene el path absoluto de todos los informes
# l_pdf es el mapeo individual de cada informe
l_path = nimgenetics.load_files(root_dir=folder_path)[0]
l_pdf = nimgenetics.load_files(root_dir=folder_path)[1]

## 3.2.2 Obtención de los identificadores reales de los informes ##

# id_peticion es una lista con los ID en formato decimal obtenidos tras la decodificación hexadecimal
# de los PDF mapeados en l_pdf
id_peticion = []
for pdf in l_pdf:
    # split por guion bajo
    s_pdf = pdf.split('_')
    # transformación del segundo componente en hexadecimal del identificador
    hex_id = int(s_pdf[1],16)
    id_peticion.append(hex_id)

# Consulta SQL utilizada para obtener finalmente los identificadores reales de la tabla correspondiente
# de la base datos la empresa
r_id = nimgenetics.getIDs(id_list=id_peticion)

## 3.2.3 Almacenamiento de los informes con su identificador correcto ##

# Generar una lista que almacene todos los identificadores provenientes de r_id
# l_names a diferencia de la anterior no es una lista de listas
l_names = []   
for row in r_id:
    try:
        # Primer bloque: extraer identificadores que cumplan la siguiente estructura
        l_names.append(row[0][0])
    except:
        # Segundo bloque: identificadores que devuelven elementos vacios
        # Estos informes se almacenarán con su identificador hexadecimal original
        s_row = str(row)
        if s_row == '[]':
            # Almacenamos en l_names los identificadores reemplazados con el literal 'sin_id'
            sr_row = s_row.replace('[]', 'sin_id')
            l_names.append(sr_row)


# Función utilizada para almacenar los informes que cumplen un patrón de selección pre-establecido
# También se almacenan los informes que están repetidos y los descartados que no cumplen el patrón
# con la finalidad de no perder información
nimgenetics.identifiedPDF(folder_path=folder_result, list_id=l_names, list_path=l_path)

# Función para almacenar los informes cuyo identificador no ha sido recuperado de la base de datos
# Estos informes son guardados con el identificador hexadecimal original
nimgenetics.emptyPDF(folder_path=folder_result, list_id=l_names, list_path=l_path, list_pdf=l_pdf)

### 3.3 Conversión de los informes a texto plano para la extracción de terminología médica ###

# Generar imagenes PNG convirtiendo PDF escaneados
# path de la carpeta Identificados donde se encuentran los PDF que cumplieron el patrón de selección
pathI = 'path_to_folder_identificados'
# Utilizar el módulo os para tratar el contenido de la carpeta como una lista
l_I = os.listdir(pathI)
# Lista que contiene el path absoluto de todos los pdf
# Esta lista es necesario para ejecutar correctamente la función pdfToImage
pdfPath = []
for pdf in l_I:
    pdfPath.append(os.path.join(pathI, pdf))
    

## 3.3.1 Generación de imágenes en formato PNG a partir de PDF escaneados ##

# Ejecución de la función pdfToImage    
nimgenetics.pdfToImage(f_path=folder_result, pdf_list=pdfPath)

## 3.3.2 Generación de texto plano a partir de imágenes en formato PNG ##

# Ejecución de la función makeCorpus la cual integra en su código el programa ocrToImage
# path donde están guardadas las imágenes
image_path = 'path_to_images'
nimgenetics.makeCorpus(f_path=folder_result, image_path=image_path)


# ------------------------------- 4. EXPERIMENTOS REALIZADOS Y RESULTADOS --------------------------------------- #

# path donde se localizan los textos
textos = 'path_to_textos'
l_textos = os.listdir(textos)
# lista que contiene el path absoluto de los textos
path_textos = []
for file in l_textos:
    path_textos.append(os.path.join(textos, file))
    
### 4.2 Extracción de terminología mediante cutext ###

# path a la carpeta in de cutext
cutext_in = 'path_to_cutext_in_folder'
# path del ouput generado por cutext
out = 'path_to_cutext_output_folder'
# path de la carpeta cutext results donde se almacena el output generado por cutext
cutext_results = 'path_to_cutext_results'

# ejecución de la función que integra la funcionalidad de cutext
nimgenetics.getCutextResults(cutext_in=cutext_in, textos_ori=path_textos, old=out, new=cutext_results)

## 4.2.1 Elaboración de un corpus final a partir de los resultados Cutext ##

# path absoluto de los resultados de cutext
cutext = os.listdir(cutext_results)
l_cutext = []
for file in cutext:
    l_cutext.append(os.path.join(cutext_results, file))
    
# path donde se guardará el corpus elaborado a partir de los resultados de cutext
cutext_corpus = 'path_to_cutext_corpus_folder'

# ejecución del programa que genera el corpus de cutext
nimgenetics.cutextCorpus(lista_cutext=l_cutext, path_to_words=cutext_corpus)